In [1]:
%load_ext autoreload
%autoreload 2

# Load data

In [3]:
from src.loader import load_and_clean

# Load, clean and covert anual subs to monthly
df, df_transactions = load_and_clean()
df_last_month = df.loc[
    (df['mrr_month'].dt.year == 2017)
    & (df['mrr_month'].dt.month == 12)
]

ModuleNotFoundError: No module named 'src'

# Distribution and size

> Each row seems to be one payment / transaction.

_How are users distributed?_

In [ ]:
from src.plot import country_order, product_order, plot_simple_pie_chart, colors_pie_product

In [ ]:
counts_aux = df.groupby(['billingCountry'])['userId'].nunique()
tot = sum(df.groupby(['billingCountry'])['userId'].nunique())
counts = (counts_aux/tot * 100).reindex(country_order)

plot_simple_pie_chart(counts, title='Nº of historical users by countries')

> US, UK and France lead the market share. India is a close 4th while Canada lags behind.

_Which country generates the most revenue?_

In [ ]:
counts = df.groupby(['billingCountry'])['mrr'].sum().reindex(country_order)
plot_simple_pie_chart(counts, title='Distribution of revenue per country (total)')

> US and UK account for 50% of the total revenue.

_How are products distributed globally?_ (last month)

In [ ]:
counts = df_last_month['product'].value_counts(normalize=True).reindex(product_order)
plot_simple_pie_chart(counts, title='Distribution of Products', color_palette=colors_pie_product)

> In the last month of data, we can see that GURU product accounts for more than half the active subscriptions

_What product brings the more revenuve globally?_ (last month)

In [ ]:
counts = df.groupby(['product'])['mrr'].sum().reindex(product_order)
plot_simple_pie_chart(counts, title='Distribution of revenue per product (total)', color_palette=colors_pie_product)

> While BUSINESS only accounts for 4.8% of total subscriptions in the last months, it represents 17% of the total revenue in that month.

# Product and period over time

In [ ]:
import pandas as pd
from src.processing import calculate_proportions
from src.plot import plot_compare, color_dict_product

In [ ]:
df_tmp = df.copy()
df_tmp_transactions = df_transactions.copy()

In [ ]:
# By product
group_cols = ['product']
hue_col = 'product'
title = 'Global'

df_active_users = calculate_proportions(df=df_tmp, date_col='mrr_month', group_cols=group_cols)
df_transactions_aux = (
    df_tmp_transactions
    .set_index('datetime')
    .groupby([pd.Grouper(freq='MS')] + group_cols)
    .size()
    .reset_index()
    .rename(columns={0: 'n', 'datetime': 'mrr_month'})
)
df_amount = df_tmp.set_index('mrr_month').groupby([pd.Grouper(freq='MS')] + group_cols)['amount'].sum().reset_index()
plot_compare(
    df_active_users=df_active_users,
    df_transactions=df_transactions_aux,
    df_amount=df_amount,
    x_col='mrr_month',
    title=title,
    hue_col=hue_col,
    hue_order=product_order,
    hue_color_dict=color_dict_product,
)
df_amount['pct'] = df_amount.groupby('mrr_month')['amount'].apply(lambda x: (x / x.sum()) * 100).values
print(df_amount.loc[df_amount['mrr_month'] == df_amount['mrr_month'].max()])


# By period
group_cols = ['period']
hue_col = 'period'

df_active_users = calculate_proportions(df=df_tmp, date_col='mrr_month', group_cols=group_cols)
df_transactions_aux = (
    df_tmp_transactions
    .set_index('datetime')
    .groupby([pd.Grouper(freq='MS')] + group_cols)
    .size()
    .reset_index()
    .rename(columns={0: 'n', 'datetime': 'mrr_month'})
)
df_amount = df_tmp.set_index('mrr_month').groupby([pd.Grouper(freq='MS')] + group_cols)['amount'].sum().reset_index()
plot_compare(
    df_active_users=df_active_users,
    df_transactions=df_transactions_aux,
    df_amount=df_amount,
    x_col='mrr_month',
    title=title,
    hue_col=hue_col,
)
df_amount['pct'] = df_amount.groupby('mrr_month')['amount'].apply(lambda x: (x / x.sum()) * 100).values
print(df_amount.loc[df_amount['mrr_month'] == df_amount['mrr_month'].max()])

> Globally, we can see that from 2017 there is a clear shift in trend from PRO to GURU products.

> At the end of 2018, we can see another shift in transactions, shifting again from GURU to PRO.

> For monthly subscriptions, the impact of transaction slowdown is immediate in the following month when observing active subscriptions. The fact that we can see a downward trend on active subscriptions (be it from PRO or GURU products) a month later already tells us that the majority of users are only subbed for one month.

> Focusing on MRR, we can see that the anual subscribers bring in a lot more revenue that monthly subscribers (in the last month, 96% of the revenue was from anual subscriptions). By product, 'GURU' brings 2/3 of the revenue in the last month.

> Again looking at MRR, GURU is the most profitable product by quite a margin.

##### Combined plot

In [ ]:
df_tmp = df.copy()
df_tmp_transactions = df_transactions.copy()

group_cols = ['product', 'period']
hue_col = 'product'
title = 'Global'

df_active_users = calculate_proportions(df=df_tmp, date_col='mrr_month', group_cols=group_cols)

df_transactions_aux = (
    df_tmp_transactions
    .set_index('datetime')
    .groupby([pd.Grouper(freq='MS')] + group_cols)
    .size()
    .reset_index()
    .rename(columns={0: 'n', 'datetime': 'mrr_month'})
)

df_amount = df_tmp.set_index('mrr_month').groupby([pd.Grouper(freq='MS')] + group_cols)['amount'].sum().reset_index()

plot_compare(
    df_active_users=df_active_users,
    df_transactions=df_transactions_aux,
    df_amount=df_amount,
    x_col='mrr_month',
    title=title,
    hue_col=hue_col,
    hue_order=product_order,
    hue_color_dict=color_dict_product,
    b_detail_period=True,
    figsize=(15,4),
    dpi=200,
)

> As expected, the dotted lines correlate by 1 month between transactions and active subscriptions

Going product by product

1. PRO:
    - Anual transactions slowdown but strong monthly uptrend. These will salvage a few month of MRR in the short term, but long term will be impacted.
2. GURU:    
    - Both anual and monthly transactions are slowing down, we can expect a drop in MRR in the following months.
3. BUSINESS:
    - Both anual and monthly transactions are also slowing down, we can also expect a dro in MRR in the following months.

# Last year

> We can focus on last year to confirm what we assesed from above

In [ ]:
df_tmp = df.loc[df['mrr_month'].dt.year == 2017].copy()
df_tmp_transactions = df_transactions.loc[df_transactions['datetime'].dt.year == 2017]

# By product
group_cols = ['product', 'period']
hue_col = 'product'
title = 'Global'

df_active_users = calculate_proportions(df=df_tmp, date_col='mrr_month', group_cols=group_cols)

df_transactions_aux = (
    df_tmp_transactions
    .set_index('datetime')
    .groupby([pd.Grouper(freq='MS')] + group_cols)
    .size()
    .reset_index()
    .rename(columns={0: 'n', 'datetime': 'mrr_month'})
)

df_amount = df_tmp.set_index('mrr_month').groupby([pd.Grouper(freq='MS')] + group_cols)['amount'].sum().reset_index()

plot_compare(
    df_active_users=df_active_users,
    df_transactions=df_transactions_aux,
    df_amount=df_amount,
    x_col='mrr_month',
    title=title,
    hue_col=hue_col,
    hue_order=product_order,
    hue_color_dict=color_dict_product,
    b_detail_period=True,
    figsize=(15,4),
    dpi=200,
)

> Focusing on last year, we can see that the number of transactions for monthly users has a peak in sept. 2017
> MRR of GURU products slowdowns from oct, which makes sense due to transactions slowding down from sept.

*This is the same plot but spearating the monthly and anual subscriptions*

In [ ]:
from src.plot import plot_anual_vs_monthly_by_product

In [ ]:
plot_anual_vs_monthly_by_product(df, df_transactions, 2017, product_order, color_dict_product)

> We can only confirm what we said earlier for each product

# Forecast

## Baseline

In [ ]:
from src.model import get_baseline_predictions, display_baseline_metrics_and_predictions
from src.plot import plot_baseline_predictions

**Predictions**

In [ ]:
mrr_monthly = df.groupby([pd.Grouper(key='mrr_month', freq='MS'), 'product'])['mrr'].sum().unstack().fillna(0)
baseline_metrics, baseline_predictions, baseline_future_predictions = get_baseline_predictions(mrr_monthly=mrr_monthly)

__Results__

In [ ]:
palette = [
    'lightblue',
    '#370031ff',
    'darkblue',
    '#ce8964ff',
    '#04E762',
]

plot_baseline_predictions(baseline_predictions, baseline_future_predictions, product_order, palette=palette)

**Metrics**

In [ ]:
display_baseline_metrics_and_predictions(baseline_metrics, baseline_future_predictions)

## SARIMAX

In [ ]:
from src.model import get_sarima_predictions, display_sarima_metrics_and_predictions
from src.plot import plot_sarima_predictions

**Predictions**

In [ ]:
mrr_monthly = df.groupby([pd.Grouper(key='mrr_month', freq='MS'), 'product'])['mrr'].sum().unstack().fillna(0)
sarima_metrics, sarima_forecasts, sarima_observed = get_sarima_predictions(mrr_monthly)

**Plot**

In [ ]:
plot_sarima_predictions(sarima_observed, sarima_forecasts)

**Metrics**

In [ ]:
display_sarima_metrics_and_predictions(sarima_metrics, sarima_forecasts, mrr_monthly)

## Prophet

In [ ]:
from src.model import get_prophet_predictions, display_metrics_and_predictions
from src.plot import plot_prophet_predictions

__Predictions__

In [ ]:
mrr_monthly = df.groupby([pd.Grouper(key='mrr_month', freq='MS'), 'product'])['mrr'].sum().unstack().reset_index()
prophet_data, prophet_metrics, prophet_forecasts = get_prophet_predictions(mrr_monthly)

__Plot__

In [ ]:
plot_prophet_predictions(prophet_data, prophet_forecasts)

__Metrics__

In [ ]:
display_metrics_and_predictions(prophet_metrics, prophet_forecasts)

# Comparison

## Metrics

In [ ]:
from src.model import get_comparative_metrics

df_metrics = get_comparative_metrics(baseline_metrics, sarima_metrics, prophet_metrics)
df_metrics.sort_values(by=['Product', 'MAE'])

Overall observations:
- Generally MAE and RMSE are relativelay close to each other, suggesting there is not variance in the errors
- ARIMA performs best for the BUSINESS and GURU category, while SMA performs slightly better than ARIMA, although the differences are not very large
- Given the simplicity of the SMA vs the ARIMA, I would be inclined to use this model instead of the ARIMA (assuming I don't have any other information available)
- PROPHET model seems so stick to the trend too much.



## Forecasts

In [ ]:
from src.model import get_comparative_forecasts

df_forecast_metrics = get_comparative_forecasts(baseline_future_predictions, prophet_forecasts, sarima_forecasts)
df_forecast_metrics

In [ ]:
df_export = df_forecast_metrics.reset_index()
df_export = df_export.loc[df_export['Model']=='Simple Moving Average']
df_export.to_csv('results/forecast.csv', sep=',', index=False)

In [ ]:
# used for quick avg calculations
# mean_aux_list = []
# for product in product_order:
#     data = sarima_observed[product]
#     mean_aux = data.loc[data.index >= '2017-01-01'].mean()
#     mean_aux_list.append(mean_aux)
#     print(f"{product} - Average from 2017: {mean_aux:.2f}")